In [4]:
import pandas as pd
import re

# from sqlalchemy.dialects.postgresql import insert

from langdetect import detect

# from . import db, genius
from __init__ import genius
import classification_2
# from .models import Songs

In [5]:
from spotipy import SpotifyClientCredentials, util
import os
import spotipy
manager = SpotifyClientCredentials(os.environ.get("SPOTIPY_CLIENT_ID"),os.environ.get("SPOTIPY_CLIENT_SECRET"))
spotify = spotipy.Spotify(client_credentials_manager=manager)

In [20]:
df = pd.read_csv("../test.csv")

In [21]:
weights = classification_2.main(df)

# df['user_id'] = my_user_id
df['weights'] = weights.tolist()
df = df.drop(['lyrics'], axis=1)

In [22]:
df

,Unnamed: 0,track_name,artists,song_id,language,weights
0,1,Suicidal,YNW Melly,1iSqfoUFnQwV0QW1EfUit8,en,"[0.7929405570030212, 0.5531405806541443, 112.1..."
1,2,New Beginnings,NEFFEX,1umyI924GDJ9cMAIQrJ0uS,en,"[0.796259343624115, 0.5454947352409363, 104.28..."
2,3,Eastside (with Halsey & Khalid),benny blanco,0d2iYfpKoM0QCKvcLCkBao,en,"[0.7913309931755066, 0.5420040488243103, 103.1..."
3,4,7 rings,Ariana Grande,14msK75pk3pA33pzPVNtBF,en,"[0.7883176207542419, 0.5498868823051453, 106.2..."
4,5,What A Man Gotta Do,Jonas Brothers,4wNIkl5XGiAACjFBlDWuSd,en,"[0.7691314816474915, 0.5354081988334656, 96.05..."
5,6,Sunflower - Spider-Man: Into the Spider-Verse,Post Malone,3KkXRkHbMCARz0aVfEt68P,en,"[0.7951471209526062, 0.5534660220146179, 108.9..."
6,7,Here Comes The Sun - Remastered 2009,The Beatles,6dGnYIeXmHdcikdzNNDMm2,en,"[0.7427708506584167, 0.5060083270072937, 97.18..."
7,8,Super Bass,Nicki Minaj,0iYebKFUSfF72fUu2OW6ZT,en,"[0.8874949812889099, 0.5766801238059998, 142.4..."
8,9,Ridin' Solo,Jason Derulo,6BaxqcoEM9r3LXisTyJjST,en,"[0.728360116481781, 0.527718722820282, 93.4618..."
9,10,Time of Our Lives,Pitbull,2bJvI42r8EF3wxjOuDav4r,en,"[0.8550240397453308, 0.5678886771202087, 128.0..."


In [14]:
import json
import numpy as np
with open('static/assets/movie_data.json') as f:
        data = json.load(f)

In [23]:
list(data.values())[0]['weights']

[0.5676284,
 1.2058978,
 -0.2627478,
 -1.1515511,
 -1.1492127,
 -0.53111666,
 -2.7649083,
 -5.876196]

In [67]:
avg_movie_weight = np.array([i['weights'] for i in data.values()]).reshape(10,8).mean(axis = 0)
weights = np.array(df['weights'].values.tolist())
avg_norms = np.linalg.norm(weights - avg_movie_weight, axis = 1)

# weights = np.array(df['weights'].values.tolist())
ix = np.argsort(avg_norms)
df.iloc[ix]
# test_df = df[avg_norms > 5.5]

AttributeError: 'TensorSliceDataset' object has no attribute 'values'

In [53]:
for chosen_movie in data:
    weights = np.array(df['weights'].values.tolist())
        # calculate songs closest in weight to movie weights
    norms = np.linalg.norm(weights - np.array(data[chosen_movie]['weights']), axis = 1,)
    ix = np.argsort(norms)
    sorted_df = df.iloc[ix]
    top_3_songs = sorted_df.iloc[0:3]['track_name'].to_list()

    print(chosen_movie, "\n", top_3_songs, norms[sorted_df.iloc[0:3].index])

Star Wars 
 ['Limelight', 'Body Like A Back Road', 'Let It Be - Remastered 2009'] [2.25215403 3.15101791 3.88351037]
Star Trek 
 ['Limelight', 'What Lovers Do (feat. SZA)', 'Are You Bored Yet? (feat. Clairo)'] [2.29312909 3.11037156 3.62229383]
Shawshank Redemption 
 ['Limelight', "Wouldn't It Be Nice - Stereo / Remastered", 'What Lovers Do (feat. SZA)'] [2.79726329 3.39505019 3.64589578]
Dead Poets Society 
 ['What Lovers Do (feat. SZA)', "Wouldn't It Be Nice - Stereo / Remastered", 'Panini - DaBaby Remix'] [3.58360847 3.76526175 3.98330732]
The Bourne Ultimatum 
 ['Limelight', 'What Lovers Do (feat. SZA)', "Wouldn't It Be Nice - Stereo / Remastered"] [2.43319676 2.48221992 3.82781866]
The Fellowship of the Ring 
 ["Wouldn't It Be Nice - Stereo / Remastered", 'Limelight', 'What Lovers Do (feat. SZA)'] [2.37501041 3.44429014 3.99558291]
Forrest Gump 
 ['What Lovers Do (feat. SZA)', 'Limelight', "Wouldn't It Be Nice - Stereo / Remastered"] [2.36117576 2.57938569 3.83490619]
Good Will Hu

In [49]:
np.linalg.norm(np.array(data["Shawshank Redemption"]['weights']) - np.array(data["Dead Poets Society"]['weights']))

2.6780088985139527

In [14]:
import pandas as pd
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow import keras 
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers.experimental.preprocessing import StringLookup

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
MAX_TOKENS = 2000
SEQUENCE_LENGTH = 500

def create_model(max_tokens=None):

    lyrics_input = keras.Input(
        shape = (500,), 
        name = "lyrics",
        dtype = "int32"
    )
    # lyrics_features = vectorize_layer(lyrics_input)
    lyrics_features = layers.Embedding(max_tokens or MAX_TOKENS, 60, name = "embedding")(lyrics_input)
    lyrics_features = layers.Dropout(0.2)(lyrics_features)
    # lyrics_features = layers.GlobalAveragePooling1D()(lyrics_features)
    # lyrics_features = layers.LSTM(100)(lyrics_features)
    # lyrics_features = layers.Dropout(0.2)(lyrics_features)
    # lyrics_features = layers.Dense(32, activation='relu')(lyrics_features)
    lyrics_features = layers.Conv1D(64, 5, activation='relu')(lyrics_features)
    lyrics_features = layers.MaxPooling1D(pool_size=4)(lyrics_features)
    lyrics_features = layers.LSTM(100)(lyrics_features)
    lyrics_features = layers.Dropout(0.2)(lyrics_features)
    lyrics_features = layers.Dense(64, activation='relu')(lyrics_features)
    lyrics_features = layers.Dense(32, activation='relu')(lyrics_features)
    output1 = layers.Dense(4, name = "energy")(lyrics_features)
    # output2 = layers.Dense(1, name = "valence")(lyrics_features)
    # output3 = layers.Dense(1, name = "tempo")(lyrics_features)
    # output4 = layers.Dense(1, name = "liveness")(lyrics_features)
    model = keras.Model(inputs=lyrics_input, outputs=[output1])
    model.compile(loss='mae',
              optimizer='adam',)
    model.load_weights('../checkpoints/my_checkpoint_30')

    return model

vectorize_layer = TextVectorization(
    max_tokens=MAX_TOKENS, # only consider this many words
    output_mode='int',
    output_sequence_length=SEQUENCE_LENGTH) 

def vectorize_movie_scripts(text):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text)

import re
def stringProcessing(s):
    s = re.sub(r"\'", "", s)
    s = re.sub(r'\n', ' ', s)
    s = re.sub(r'\t', '', s)
    s = re.sub(r"\[[^[]*\]", '', s)
    s = re.sub(r'[^\w\s]', ' ', s)
    s = re.sub(r' +', ' ', s)
    s = s.strip()
    s = s.lower()
    return s

In [23]:
import os
model = create_model() 

# df = pd.read_csv("https://raw.githubusercontent.com/benbrill/MoodSpace/main/data/trainingSongs_clean.csv")

# data = tf.data.Dataset.from_tensor_slices((df["lyrics"]))
# data_vec = data.map(vectorize_headline)
# df = pd.DataFrame(model.predict(data_vec))
# df.shape
d = {}
for script_path in os.listdir("../scripts"):
    with open(f"../scripts/{script_path}") as f:
        contents = f.read()

        contents = stringProcessing(contents)

        df2 = pd.DataFrame({"lyrics": [contents]})
        


        data = tf.data.Dataset.from_tensor_slices((df2["lyrics"]))
        vectorize_layer.adapt(data)
        data_vec = data.map(vectorize_movie_scripts)
        d[script_path] = model.predict(data_vec)[0]
        print(script_path, model.predict(data_vec)[0])

bourne.txt [  0.8028168    0.54838055 108.58993      0.13968945]
deadpoets.txt [  0.91196984   0.58987993 151.97827      0.16713756]
fellowship.txt [ 0.73340935  0.5333094  89.36578     0.13055968]
forrest.txt [  0.8148412    0.5624935  116.31569      0.13917923]
goodwillhunting.txt [  0.8513357    0.5705814  132.71431      0.16935092]
incredibles.txt [  0.81596416   0.5668687  122.77204      0.13890487]
jedi.txt [  0.7981779    0.5339753  110.80746      0.15427852]
khan.txt [  0.8221025    0.5758075  130.59529      0.14089489]
shawshank.txt [  0.81988424   0.5672764  121.83119      0.14175367]
titanic.txt [  0.8225643    0.5699205  123.32264      0.14206994]


In [28]:
for key in d.keys():
    weights = np.array(df['weights'].values.tolist())
        # calculate songs closest in weight to movie weights
    norms = np.linalg.norm(weights - d[key], axis = 1,)
    ix = np.argsort(norms)
    sorted_df = df.iloc[ix]
    top_3_songs = sorted_df.iloc[0:6]['track_name'].to_list()

    print(key, "\n", top_3_songs, norms[sorted_df.iloc[0:6].index])

bourne.txt 
 ['My Type', 'Sunflower - Spider-Man: Into the Spider-Verse', 'Paradise', 'Still The One', 'Rodeo', 'SAD!'] [0.27919775 0.40932144 0.94714614 1.14510073 1.28670679 2.32040495]
deadpoets.txt 
 ['Animal', 'Let It Be - Remastered 2009', 'Silence', 'Learning To Fly', 'Super Bass', 'Give Me Everything (feat. Ne-Yo, Afrojack & Nayer)'] [ 5.30707812  7.53202039  7.68696145  7.85513245  9.49596739 11.71960563]
fellowship.txt 
 ['SICKO MODE', "Ridin' Solo", "Wouldn't It Be Nice - Stereo / Remastered", 'Falling', 'Needs', 'What A Man Gotta Do'] [0.93414801 4.09612638 4.30051829 5.29299471 6.60576203 6.6892935 ]
forrest.txt 
 ['Adore You', 'Rather Be (feat. Jess Glynne)', 'End Of The Line', 'Talk (feat. Disclosure)', 'Everybody Wants To Rule The World', 'Suicidal'] [0.66819615 1.47093293 1.87153732 2.13546569 2.4059437  4.15976671]
goodwillhunting.txt 
 ['VIBEZ', 'Across The Universe - Remastered 2009', 'What Lovers Do (feat. SZA)', 'Palm Trees', 'Limelight', 'Time of Our Lives'] [0.2

In [17]:
import numpy as np

In [19]:
df

,lyrics
0,jack and rose drift under the blazing stars th...
